# AR(1) + GARCH(1,1) Model

## Load Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Loading Data

In [3]:
xx  = readdlm("Data/FFdSizePs.csv",',',header=true)
xx  = xx[1]
ymd = xx[:,1]     #[YearMonthDay, eg. 20121231]
R   = xx[:,2]     #returns for the smallest size portfolio
xx  = nothing

y = R[2:end]                           #dependent variable, y(t)
x = [ones(size(R,1)-1) R[1:end-1]]     #regressors, [1, y(t-1)]

8324×2 Array{Float64,2}:
 1.0  -1.18
 1.0  -1.37
 1.0   1.73
 1.0   0.77
 1.0   1.41
 1.0   0.68
 1.0   1.26
 1.0   0.61
 1.0   1.11
 1.0   0.63
 1.0   0.64
 1.0   0.37
 1.0   0.49
 ⋮         
 1.0  -0.52
 1.0  -0.3 
 1.0   0.18
 1.0   1.13
 1.0   1.11
 1.0  -0.11
 1.0   0.58
 1.0  -0.53
 1.0  -0.57
 1.0  -0.45
 1.0  -0.35
 1.0  -0.34

## The Likelihood Function

Consider a regression equation, where the residual follows a GARCH(1,1) process

$$
\begin{align*}
y_{t}           &  =x_{t}^{\prime}b+u_{t} \ \text{ with }\ u_{t}=v_{t}\sigma_{t}\\
\sigma_{t}^{2}  &  =\omega+\alpha u_{t-1}^{2}+\beta\sigma_{t-1}^{2}.
\end{align*}
$$
Note that we require $(\omega,\alpha,\beta)$ to all be positive and $\alpha + \beta < 1$.

If $v_{t}\sim N(0,1)$, then the likelihood function is

$$
\ln L=-\frac{T}{2}\ln\left(  2\pi\right)  -\frac{1}{2}\sum_{t=1}
^{T}\ln\sigma_{t}^{2}-\frac{1}{2}\sum_{t=1}^{T}\frac{u_{t}^{2}}{\sigma_{t}
^{2}}.
$$

The likelihood function of a GARCH(1,1) model can be coded as in
*garch11LL*. The first function calculates time-varying  variances and
the likelihood contributions (for each period). The second functions forms the
loss function used in the minimization.

In [4]:
function garch11LL(par::Vector,y,x)

  (T,k) = size(x,1,2)

  b                   = par[1:k]             #mean equation, y = x'*b
  (omega,alpha,beta1) = par[k+1:k+3]         #GARCH(1,1) equation: 
                                             #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)        
  yhat = x*b
  u    = y - yhat
  s2_0 = var(u)                                 #var(u,1) gives a matrix, var(u) a scalar

  s2    = zeros(typeof(alpha),T)                 #works also with ForwardDiff 
  s2[1] = omega + alpha*s2_0 + beta1*s2_0        #simple, but slow approach
  for t = 2:T                                    #using filter() is perhaps quicker
    s2[t] = omega + alpha*u[t-1]^2 + beta1*s2[t-1]
  end

  LL    = -(1/2)*log(2*pi) - (1/2)*log.(s2) - (1/2)*(u.^2)./s2
  LL[1] = 0               #effectively skip the first observation

  return LL,s2,yhat

end


function garch11LLLoss(par::Vector,y,x)
    par1            = copy(par)
    par1[end-2:end] = abs.(par1[end-2:end])     #impose non-negativity
    LL,  = garch11LL(par1,y,x)
    Loss = -sum(LL)                             #to minimize: -sum(LL)
    return Loss
end


function garch11LLRLoss(par::Vector,y,x,rho)     #with restriction on alpha+beta1 < 1
    par1            = copy(par)
    par1[end-2:end] = abs.(par1[end-2:end])     #impose non-negativity
    LL, = garch11LL(par1,y,x)
    (alpha,beta1) = par1[end-1:end]   #s2(t) = omega + alpha*u(t-1)^2 + beta1*s2(t-1)
    g = [alpha + beta1 - 1]                     #alpha + beta1 < 1
    Loss = -sum(LL) + rho*sum(max.(0,g).^2)
    return Loss
end

garch11LLRLoss (generic function with 1 method)

## Maximize the Likelihood Function

In [5]:
#mean equation, y = x'b
#GARCH(1,1) equation: s2(t) = omega + alfa*u(t-1)^2 + beta1*s2(t-1)

par0 = [mean(y);0;var(y)*0.05;0.05;0.93]         #initial parameter guess

(loglik,s2,yhat) = garch11LL(par0,y,x)            #just testing the log lik
LL               = garch11LLLoss(par0,y,x)
printlnPs("Value of log-likelihood fn at starting guess: ",-LL)

Value of log-likelihood fn at starting guess:  -9846.688


In [6]:
Sol = optimize(par->garch11LLLoss(par,y,x),par0)   #do MLE by optimization with optimize, minimize -sum(LL)

parHat            = Optim.minimizer(Sol)          #extract the optimal solution 
parHat[end-2:end] = abs.(parHat[end-2:end])       #since the likelihood function uses abs(these values)

LLHat             = garch11LLLoss(parHat,y,x)
printlnPs("Value of log-likelihood fn at estimate: ",-LLHat)

println("\nParameter estimates (b[1],b[2],omega,alpha,beta1): ")
printmat(parHat)

Value of log-likelihood fn at estimate:  -8243.608

Parameter estimates (b[1],b[2],omega,alpha,beta1): 
     0.049
     0.256
     0.014
     0.162
     0.824



## Standard Errors of the Estimates

MLE is typically asymptotically normally distributed
$$
\begin{align*}
\sqrt{T}(\hat{\theta}-\theta)  &  \rightarrow^{d}N(0,V)\text{, where } V=I(\theta)^{-1}\text{ with }\\
I(\theta)  &  =-\operatorname*{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}}
\end{align*}
$$
where $\ln L_t$  is the contribution of period $t$ to the likelihood function and $I(\theta)$ is the information matrix.

The code below calculates numerical derivatives.  


Alternatively, we can use the outer product of the gradients to calculate the
information matrix as
$$
J(\theta)=\operatorname*{E}\left[  \frac{\partial\ln L_t}{\partial\theta
}\frac{\partial\ln L_t}{\partial\theta^{\prime}}\right]
$$

We could also use the "sandwich" estimator

$$
V=I(\theta)^{-1}J(\theta)I(\theta)^{-1}.
$$

In [7]:
T = size(y,1)                    #finding std(coefs) by inverse of information matrix

Ia         = -ForwardDiff.hessian(par->mean(garch11LL(par,y,x)[1]),parHat)
Ia         = (Ia+Ia')/2              #to guarantee symmetry
vcv        = inv(Ia)/T
std_parHat = sqrt.(diag(vcv))

println("std from Hessian")
printmat(std_parHat)

std from Hessian
     0.006
     0.012
     0.001
     0.009
     0.009



In [8]:
LLgrad = ForwardDiff.jacobian(par->garch11LL(par,y,x)[1],parHat)   #T x length(par) matrix, T gradients
J           = LLgrad'LLgrad/T
vcv         = inv(J)/T
stdb_parHat = sqrt.(diag(vcv))                          #std from gradients

vcv         = inv(Ia) * J * inv(Ia)/T
stdc_parHat = sqrt.(diag(vcv))                          #std from sandwich

printlnPs("\nGARCH parameter estimates and 3 different standard errors
    coef     hessian     gradient  sandwich")
printmat([parHat std_parHat stdb_parHat stdc_parHat])


GARCH parameter estimates and 3 different standard errors
    coef     hessian     gradient  sandwich
     0.049     0.006     0.006     0.007
     0.256     0.012     0.012     0.013
     0.014     0.001     0.001     0.003
     0.162     0.009     0.005     0.020
     0.824     0.009     0.005     0.019



In [9]:
println("\nIterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED")

options = Optim.Options(show_trace=false,show_every=10)
for rho = 0.0:0.1:1
  Sol = optimize(par->garch11LLRLoss(par,y,x,rho),par0,options)
  par = Optim.minimizer(Sol)
  par[end-3:end-1] = abs.(par[end-3:end-1])
  printmat([rho par'])
  par0 = deepcopy(par)
end


Iterate with harder and harder punishment on α + β < 1 restriction. PERHAPS NOT NEEDED
     0.000     0.049     0.256     0.014     0.162     0.824

     0.100     0.049     0.256     0.014     0.162     0.824

     0.200     0.049     0.256     0.014     0.162     0.824

     0.300     0.049     0.256     0.014     0.162     0.824

     0.400     0.049     0.256     0.014     0.162     0.824

     0.500     0.049     0.256     0.014     0.162     0.824

     0.600     0.049     0.256     0.014     0.162     0.824

     0.700     0.049     0.256     0.014     0.162     0.824

     0.800     0.049     0.256     0.014     0.162     0.824

     0.900     0.049     0.256     0.014     0.162     0.824

     1.000     0.049     0.256     0.014     0.162     0.824



## Value at Risk

In [10]:
(_,s2,ER) = garch11LL(parHat,y,x)
VaR95     = -(ER - 1.64*sqrt.(s2))

CovRatio = mean((-y) .>= VaR95)             #coverage ratio for VaR
printlnPs("\nCoverage ratio for VaR(95%): ",CovRatio)


Coverage ratio for VaR(95%):      0.059


In [12]:
y   = floor.(ymd/10000)                        #converting 20121231 to Date
m   = floor.( (ymd-y*10000)/100 )
d   = ymd - y*10000 - m*100

dN = Array{Date}(length(ymd))
for t = 1:length(ymd)
    dN[t] = Date(y[t],m[t],d[t])
end      


xTicks = [Date(1990);Date(2000);Date(2010)]     #controlling the tick locations
plot(dN[2:end],VaR95,xticks=Dates.value.(xTicks),legend=false)
title!("1-day VaR (95%)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1990-01-01 
 
 
 2000-01-01 
 
 
 2010-01-01 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 1-day VaR (95%) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,317.654 39.4159,313.663 39.5536,328.285 39.5994,325.175 39.6453,330.66 39.6912,329.191 39.737,334.519 39.8746,333.34 39.9205,338.357 39.9664,338.056 
 40.0122,341.333 40.0581,342.602 40.1957,346.48 40.2416,349.759 40.2874,333.243 40.3333,345.344 40.3792,347.693 40.5168,349.219 40.5626,351.794 40.6085,340.433 
 40.6544,349.114 40.7003,346.716 40.8379,351.059 40.8837,351.24 40.9296,354.573 40.9755,358.075 41.0213,359.405 41.1589,359.478 41.2048,347.081 41.2507,353.581 
 41.2965,356.165 41.3424,338.217 41.5259,342.484 41.5717,343.775 41.6176,353.242 41.6635,344.238 41.8011,344.696 41.847,337.159 41.8928,349.393 41.9387,348.304 
 41.9846,352.356 42.1222,356.37 42.168,345.388 42.2139,340.153 42.2598,326.84 42.3056,287.931 42.4432,300.558 42.4891,289.929 42.535,316.476 42.5808,317.484 
 42.6267,321.438 42.7643,326.465 42.8102,275.281 42.8561,297.088 42.9019,317.511 42.9478,311.374 43.0854,317.689 43.1313,269.243 43.1771,277.227 43.223,299.89 
 43.2689,149.857 43.4065,214.84 43.4523,209.05 43.4982,221.694 43.5441,238.804 43.7275,242.725 43.7734,232.683 43.8193,261.737 43.8651,278.221 43.911,288.44 
 44.0486,293.534 44.0945,289.343 44.1404,300.826 44.1862,303.388 44.2321,304.975 44.3697,319.402 44.4156,310.2 44.4614,323.332 44.5073,319.378 44.5532,324.683 
 44.6908,317.274 44.7366,328.203 44.7825,333.641 44.8284,336.752 44.8742,335.483 45.0118,342.636 45.0577,346.59 45.1036,349.82 45.1495,353.072 45.1953,346.985 
 45.3329,348.914 45.3788,352.15 45.4247,357.187 45.4705,358.582 45.5164,358.339 45.654,359.238 45.6999,360.226 45.7457,360.399 45.7916,360.772 45.8375,363.288 
 46.0209,361.944 46.0668,358.2 46.1127,359.217 46.1586,351.278 46.2962,357.802 46.342,356.995 46.3879,358.034 46.4338,361.653 46.4796,359.531 46.6172,360.828 
 46.6631,360.689 46.709,362.11 46.7548,364.047 46.8007,357.973 46.9383,362.514 46.9842,360.414 47.03,361.886 47.0759,361.993 47.1218,359.05 47.2594,362.231 
 47.3053,365.153 47.3511,365.576 47.397,365.217 47.4429,360.122 47.5805,362.997 47.6263,347.98 47.6722,356.547 47.7181,360.07 47.9015,360.491 47.9474,357.712 
 47.9933,355.714 48.0391,360.005 48.085,355.967 48.2226,360.468 48.2685,359.284 48.3143,353.887 48.3602,356.792 48.4061,357.727 48.5437,356.387 48.5896,356.289 
 48.6354,349.884 48.6813,355.052 48.7272,358.561 48.8648,356.307 48.9106,360.08 48.9565,362.365 49.0024,361.602 49.0482,352.383 49.1858,357.89 49.2317,354.005 
 49.2776,358.374 49.3234,360.68 49.3693,360.419 49.5069,357.652 49.5528,359.586 49.5987,348.909 49.6445,355.825 49.6904,355.992 49.828,352.758 49.8739,329.54 
 49.9197,339.236 49.9656,348.172 50.0115,348.64 50.1491,346.136 50.1949,343.662 50.2408,348.142 50.2867,347.439 50.3325,343.977 50.516,352.388 50.5619,353.858 
 50.6078,353.762 50.6536,346.945 50.7912,351.708 50.8371,344.699 50.883,352.429 50.9288,355.101 50.9747,354.164 51.1123,353.668 51.1582,353.009 51.204,355.563 
 51.2499,354.897 51.2958,346.63 51.4334,351.998 51.4792,347.025 51.5251,333.086 51.571,344.601 51.6168,345.073 51.7545,314.1 51.8003,284.83 51.8462,314.242 
 51.8921,314.729 51.9379,321.505 52.0755,327.984 52.1214,330.342 52.1673,326.544 52.2131,334.976 52.259,334.926 52.3966,339.751 52.4425,344.904 52.4883,342.491 
 52.5342,347.537 52.5801,334.036 52.7177,343.799 52.7636,348.378 52.8094,345.87 52.8553,353.271 52.9012,335.09 53.0388,347.042 53.0846,335.356 53.1305,341.113 
 53.1764,349.243 53.2222,340.08 53.3598,349.151 53.4516,352.701 53.4974,354.77 53.5433,326.027 53.6809,337.561 53.7268,342.203 53.7726,349.254 53.8185,349.637 
 53.8644,348.203 54.002,347.719 54.0479,344.345 54.0937,350.245 54.1396,350.759 54.1855,353.827 54.3231,346.107 54.3689,342.742 54.4148,352.058 54.5065,